In [1]:
import unibox as ub
from s5cmdpy import S5CmdRunner
import os
runner = S5CmdRunner()


def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_chunked_list(lst, total_jobs):
    chunks = list(chunk_list(lst, len(lst) // total_jobs))
    return chunks

test: 大量文件的resize:
- 旧版uni_resizer在有大量todo文件的时候会hang up

In [2]:
TARGET_NUM = 1  # Set to the specific subset number to process
LOCAL_DEST = f"/data/subset_10m_part_{TARGET_NUM}"  # Download location
S3_DEST = "s3://dataset-pixiv/resized_1572864/".rstrip("/")

# Download the first chunk: around 170G
chunk_local_dest = f"{LOCAL_DEST}"

download benchmark images:

In [4]:
# # Load the parquet data and split into chunks of S3 URIs
# df = ub.loads(f"s3://dataset-pixiv/subsets/full_pixiv_better_10m_v3/subset_10m_part_{TARGET_NUM}.parquet")
# uris = df["s3_uri"].tolist()
# total_chunks = 10
# uri_chunks = list(chunk_list(uris, len(uris) // total_chunks))
# test_uris = uri_chunks[0]


# print(ub.peeks(test_uris))
# runner.download_from_s3_list(test_uris, chunk_local_dest)

2024-05-08 16:48:52,974 [INFO] UniLogger: UniLoader.loads: .parquet LOADED from "/tmp/tmpb1l3x76y/subset_10m_part_1.parquet" in 6.94s


{'metadata': {'len': 92903, 'item_type': 'str'}, 'preview': ['s3://dataset-pixiv/artworks/66180793/66180793_p0.jpg', 's3://dataset-pixiv/artworks/66180800/66180800_p0.jpg', 's3://dataset-pixiv/artworks/66180820/66180820_p0.png']}


[interval=5] running s5cmd:   0%|          | 0/92903 [00:00<?, ?it/s]

test resize:

In [4]:
RESIZE_DEST = f"{LOCAL_DEST}_resized"

# do the resize on the whole thing: (which would likely hang)
resizer = ub.UniResizer(
    root_dir=chunk_local_dest,
    dst_dir=RESIZE_DEST,
    target_pixels=int(1024 * 1024 * 1.5),
)

# Resize the images
images_to_resize = resizer.get_resize_jobs()
resizer.execute_resize_jobs(images_to_resize)

2024-05-08 17:21:15,414 [INFO] UniLogger: UniResizer.get_resize_jobs: Getting image paths...
2024-05-08 17:21:16,287 [INFO] UniLogger: UniResizer.get_resize_jobs: Checking existing files...
2024-05-08 17:21:18,496 [INFO] UniLogger: UniResizer.execute_resize_jobs: Resizing 75205 images from [/data/subset_10m_part_1] to [/data/subset_10m_part_1_resized]; chunk size: 10000 | report interval: 5s


Resizing 75205 images


[interval=5] resizing [/data/subset_10m_part_1]:   0%|          | 0/75205 [00:00<?, ?it/s]